In [1]:
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from alexnet import AlexNet
import time

# TODO: Load traffic signs data.
with open('train.p', mode='rb') as f:
	train = pickle.load(f)

X_train, y_train = train['features'], train['labels']
class_num = 43
print(X_train.shape)

(39209, 32, 32, 3)


In [2]:
# TODO: Split data into training and validation sets.
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25)
# TODO: Define placeholders and resize operation.
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
resized = tf.image.resize_images(x, (227,227))
y = tf.placeholder(tf.int32, (None))
y_one_hot = tf.one_hot(y, class_num)

# TODO: pass placeholder as first argument to `AlexNet`.
fc7 = AlexNet(resized, feature_extract=True)
# NOTE: `tf.stop_gradient` prevents the gradient from flowing backwards
# past this point, keeping the weights before and up to `fc7` frozen.
# This also makes training faster, less work to do!
fc7 = tf.stop_gradient(fc7)

In [3]:
# TODO: Add the final layer for traffic sign classification.
shape = (fc7.get_shape().as_list()[-1], class_num)
mu = 0
sigma = 0.1
EPOCHS = 10
BATCH_SIZE = 128

wfc8 = tf.Variable(tf.truncated_normal(shape = shape, mean = mu, stddev = sigma))
bfc8 = tf.Variable(tf.zeros(class_num))
logits = tf.matmul(fc7, wfc8) + bfc8

In [4]:
# TODO: Define loss, training, accuracy operations.
# HINT: Look back at your traffic signs project solution, you may
# be able to reuse some the code.
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_one_hot, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer()
train_oper = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y_one_hot,1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
# TODO: Train and evaluate the feature extraction model.
def evaluate(x_data, y_data, sess):
	num_example = len(x_data)
	total_accu = 0
	for i in range(0, num_example, BATCH_SIZE):
        
		batch_x, batch_y = x_data[i:i+BATCH_SIZE], y_data[i:i+BATCH_SIZE]
		accuracy = sess.run(accuracy_operation, feed_dict={x:batch_x, y:batch_y})
		total_accu += accuracy * len(batch_x)

	return total_accu/num_example

In [9]:
saver = tf.train.Saver()

# Train 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('training...')
    for epoch in range(EPOCHS):
        time1 = time.time()
        for i in range(0, X_train.shape[0], BATCH_SIZE):
            batch_x, batch_y = X_train[i:i+BATCH_SIZE], y_train[i:i+BATCH_SIZE]
            sess.run(train_oper, feed_dict={x:batch_x, y:batch_y})
        time2 = time.time()
        valid_acc = evaluate(X_valid, y_valid, sess)

        print('EPOCH: {}'.format(epoch +1), 'Validation Accuracy: ', valid_acc, 'time: ', time2-time1)

    saver.save(sess,'./trained_model')
    print('model saved.')

# Test
# with tf.Session() as sess:
# 	saver.restore(sess, tf.train.latest_checkpoint('.'))
# 	prediction = sess.run(tf.argmax(logits, 1), feed_dict={x:x})

training...
EPOCH: 1 Validation Accuracy:  0.769152300669 time:  80.42153906822205
EPOCH: 2 Validation Accuracy:  0.861674997316 time:  80.404057264328
EPOCH: 3 Validation Accuracy:  0.899418545209 time:  80.41209292411804
EPOCH: 4 Validation Accuracy:  0.918290319059 time:  80.40559911727905
EPOCH: 5 Validation Accuracy:  0.931551565616 time:  80.42961192131042
EPOCH: 6 Validation Accuracy:  0.938386207975 time:  80.3940486907959
EPOCH: 7 Validation Accuracy:  0.946036927046 time:  80.42500615119934
EPOCH: 8 Validation Accuracy:  0.949709272246 time:  80.4218602180481
EPOCH: 9 Validation Accuracy:  0.953279608216 time:  80.42797350883484
EPOCH: 10 Validation Accuracy:  0.957462001361 time:  80.41802191734314
model saved.
